In [2]:
import duckdb

In [3]:
con = duckdb.connect(database=':memory:')

In [19]:
con.execute("drop table my_table")

In [21]:
con.execute("CREATE TABLE my_table AS "
            "SELECT id, concat(given_name, ' ', surname) as name "
            "FROM 'data/S1_clean_.csv'")

In [22]:
con.execute("select * from my_table")
con.fetchall()

[('S1_0', 'joshua morrison'),
 ('S1_1', 'jordan white'),
 ('S1_2', 'emmerson lock'),
 ('S1_3', 'alexandra grosser'),
 ('S1_4', 'michael wuchatsch'),
 ('S1_5', 'emmerson loyck'),
 ('S1_6', 'rhys schuetz'),
 ('S1_7', 'joshua greenj'),
 ('S1_8', 'olivia hobson'),
 ('S1_9', 'michael lierach'),
 ('S1_10', 'elisabett domiten'),
 ('S1_11', 'genoveffa hylander')]

In [34]:
con.execute("drop sequence N")
con.execute("create sequence N start 1")

In [ ]:
# con.execute("create table incr")
# con.execute("insert into incr values(nextval('N'))")

In [ ]:
# con.execute("select id, currval('N') as pos, substring(concat(substring('#..#', 1, 3-1), "
#             "lower(name), "
#             "substring('#..#', 1, 3-1)),"
#             "nextval('N'), 3) as qgram "
#             "from my_table "
#             "where id = 'S1_0'").fetchall()